In [ ]:
from network import WLAN, STA_IF
from parameters import *
from time import sleep
wlan = WLAN(STA_IF)
wlan.active(True)
wlan.connect(WIFI_SSID, WIFI_PASSWORD)            
while not wlan.isconnected():
    sleep(4)


In [44]:
def limitedGetRequest(url, tagsToKeep=(), timeout=15):
    
    import ussl, usocket

    scheme, _, host, target = url.split("/",3)

    if scheme == "https:":
        port = 443
    elif scheme == "http:":
        port = 80

    ai = usocket.getaddrinfo(host, port, 0, usocket.SOCK_STREAM)[0]
    s = usocket.socket(ai[0], usocket.SOCK_STREAM, ai[2])
    s.settimeout(timeout) # time for connection to timeout in seconds
    s.connect(ai[-1])
    s = ussl.wrap_socket(s, server_hostname=host)
    s.write(bytes(f"GET /{target} HTTP/1.0\r\n", "utf-8"))
    s.write(bytes(f"Host: {host}\r\n", "utf-8"))
    s.write(b"User-agent: Mozilla/5.0\r\n")
    s.write(b"Connection: close\r\n\r\n")

    keptLines = []
    stringBuffer = b""
    buildBuffer = False
    while True:

        x = s.read(1)
        if x == None or x == b"":
            break
        
        if buildBuffer:
            stringBuffer += x

        if x == b"<" and not buildBuffer:
            buildBuffer = True
            stringBuffer += x
        elif x == b">" and buildBuffer:
            
            bufferTagName = stringBuffer[1:-1].split(b" ",1)[0]
            if any( tag in bufferTagName for tag in tagsToKeep ) and stringBuffer[1] != b"/":
                
                closingTag = b"</" + bufferTagName + b">"
                closingTagLen = len(closingTag)

                while buildBuffer:

                    x = s.read(1).decode()
                    if x == None or x == b"":
                        break
                    stringBuffer += x
                    if len(stringBuffer) >= closingTagLen:
                        if stringBuffer[-closingTagLen:] == closingTag:
                            print("bingo")
                            keptLines.append(stringBuffer)
                            buildBuffer = False
                            stringBuffer = b""
            else:
                stringBuffer = b""
                buildBuffer = False


    return (keptLines)

from parameters import *
url = f"https://forecast.weather.gov/MapClick.php?lat={lat}&lon={long}&FcstType=digitalDWML"
print(url)
tagsToKeep = [b"start-valid-time", b"temperature", b"probability-of-precipitation", b"cloud-amount"]
print(limitedGetRequest(url, tagsToKeep, timeout=15))


"b'A'"

In [ ]:
while True

In [73]:
bufferTagName = b'wind-speed'
tagsToKeep = [b'wind-speed', b"temperature", b"probability-of-precipitation", b"cloud-amount"]
any( tag in bufferTagName for tag in tagsToKeep )

True

'</start-valid-time>'

In [64]:
from requests import get
response = get("https://forecast.weather.gov/MapClick.php?lat=41.9295&lon=-87.7084&FcstType=digitalDWML")

In [66]:
for x in response.text.split("\n"):
    print(x)

<?xml version="1.0" encoding="ISO-8859-1"?>
<dwml version="1.0" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://graphical.weather.gov/xml/DWMLgen/schema/DWML.xsd">
  <head>
    <product concise-name="tabular-digital" operational-mode="developmental" srsName="WGS 1984">
      <creation-date refresh-frequency='PT1H'>2024-01-27T15:57:18-06:00</creation-date>
    </product>
    <source>
      <production-center>Chicago, IL</production-center>
      <credit>https://www.weather.gov/lot</credit>
      <more-information>https://www.nws.noaa.gov/forecasts/xml/</more-information>
    </source>
  </head>
  <data>

  <location>
    <location-key>point1</location-key>
    <point latitude="41.92" longitude="-87.7"/>
    <area-description>5 Miles NE Oak Park IL</area-description>
    <height datum="mean sea level" height-units="feet">604</height>
  </location>
<moreWeatherInformation applicable-location="point1">